{/* cspell:ignore Lipinska autoplay Tilly Ardle webkitallowfullscreen allowfullscreen frameborder referrerpolicy */}

# The Ansatz

Watch as Victoria Lipinska explains what an ansatz is, and why we care about it in the context of VQE.

<iframe width="780" height="438" src="https://video.ibm.com/embed/recorded/132414879?showtitle=false&#x26;forced-quality=high"></iframe>

**References**

* [The theory of variational hybrid quantum-classical algorithms, McClean, et al.](https://iopscience.iop.org/article/10.1088/1367-2630/18/2/023023/meta)
* [Quantum Chemistry in the Age of Quantum Computing, Cao, et al.](https://arxiv.org/pdf/1812.09976.pdf)
* [Noisy intermediate-scale quantum (NISQ) algorithms, Bharti, et al.](https://arxiv.org/pdf/2101.08448.pdf)
* [The Variational Quantum Eigensolver: A review of methods and best practices, Tilly, et al.](https://arxiv.org/abs/2111.05176)
* [Hardware-efficient Variational Quantum Eigensolver for Small Molecules and Quantum Magnets, Kandala, et al.](https://arxiv.org/pdf/1704.05018.pdf)
* [Quantum computational chemistry, McArdle, et al.](https://arxiv.org/pdf/1808.10402.pdf)

## Ansatz Code

In the previous lesson you created a __Hamiltonian__ describing the energy of your molecule of interest, and mapped it to a format useful for a quantum computer. The variational quantum eigensolver uses a __variational circuit__ to prepare quantum states. We then use those states to determine the expectation value of the Hamiltonian (the energy). Parameters in the variational circuit are varied until the calculation converges on a minimum expectation value. In this quantum chemistry context this should be the ground state energy. This lesson focuses on the variational circuit, also called the __ansatz__ (a German word meaning "approach" or "method").
In this lesson you will learn
- the set of pre-made ansaetze available in the circuit library
- how to specify or modify the features of an ansatz
- how to build your own ansatz
- examples of good and bad ansaetze

The Qiskit circuit library has many categories of circuit that can be used as an ansatz. Here, we will restrict our discussion to 2-local circuits (circuits consisting of gates that act on, at most, two qubits at a time). __EfficientSU2__ is a commonly-used ansatz.

An EfficientSU2 circuit consists of layers of single qubit operations spanned by SU(2) (special unity group of degree 2, like Pauli rotation gates) and CX entanglements. This is a heuristic pattern that can be useful in variational quantum algorithms like VQE and classification circuits in quantum machine learning (QML).

We'll start with a 4-qubit example EfficientSU2 circuit with two types of SU(2) gates, say rx & ry. We also specify an entanglement scheme, and the number of repetitions. If you simply .draw() the circuits, you will get a fairly abstract representation. A more comprehensible circuit diagram is obtained using .decompose().draw(), and here we will use the output = 'mpl'.

In [46]:
from qiskit.circuit.library import EfficientSU2

SU2_ansatz = EfficientSU2(4, su2_gates=["rx", "y"], entanglement="linear", reps=1)
print(SU2_ansatz.draw())
SU2_ansatz.decompose().draw(output="mpl")

     ┌────────────────────────────────────────────────────────┐
q_0: ┤0                                                       ├
     │                                                        │
q_1: ┤1                                                       ├
     │  EfficientSU2(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7]) │
q_2: ┤2                                                       ├
     │                                                        │
q_3: ┤3                                                       ├
     └────────────────────────────────────────────────────────┘


<Image src="/learning/images/courses/quantum-chem-with-vqe/ansatz/extracted-outputs/6028e7e8-7f4e-45d1-b08b-c8d4e567c1a3-1.avif" alt="Output of the previous code cell" />

The SU(2) gates appear at the beginning and end with the order and elements as specified in su2_gates = [...]. The entanglement scheme "linear" means that the CX gates step through the numbered qubits, entangling 0 & 1, then 1 & 2, and so on down a diagonal line in the cirucit. As you might expect, setting reps = 2 simply adds an entanglement layer and ending SU(2) layer. Settings reps = n corresponds to n entanglement layers, with SU(2) layers between them, and on each end.

In [47]:
SU2_ansatz2 = EfficientSU2(4, su2_gates=["rx", "y", "z"], entanglement="linear", reps=2)
SU2_ansatz2.decompose().draw(output="mpl")

<Image src="/learning/images/courses/quantum-chem-with-vqe/ansatz/extracted-outputs/e7eb387b-8766-4f3c-a0cc-f07b4b6d5e85-0.avif" alt="Output of the previous code cell" />

There are several other entanglement schemes. Two worth noting are "circular", and "full". Circular entanglement is identical to linear entanglement, but with an additional CX gate entangling the first and last qubit. The full entanglement scheme includes a CX gate between every pair of qubits. Note that for an N-qubit circuit, this is $N(N-1)/2$  $CX$ gates, and can become computationally expensive.

In [48]:
SU2_ansatz3 = EfficientSU2(
    4, su2_gates=["rx", "y", "z"], entanglement="circular", reps=1
)
SU2_ansatz3.decompose().draw(output="mpl")

<Image src="/learning/images/courses/quantum-chem-with-vqe/ansatz/extracted-outputs/b2cc6b8a-c6e8-456f-ac0c-1f8b33ef964b-0.avif" alt="Output of the previous code cell" />

In [49]:
SU2_ansatz4 = EfficientSU2(4, su2_gates=["rx", "y", "z"], entanglement="full", reps=1)
SU2_ansatz4.decompose().draw(output="mpl")

<Image src="/learning/images/courses/quantum-chem-with-vqe/ansatz/extracted-outputs/60f394ba-f338-4be9-83b4-18767c813886-0.avif" alt="Output of the previous code cell" />

You can keep an eye on your circuit depths using .depth(), but note that just as in the visualization, it is important to .decompose() first. Otherwise, the .depth() will count the entire ansatz as a single circuit element.

In [50]:
print(SU2_ansatz4.decompose().depth())
print(SU2_ansatz4.depth())

11
1


A generalization of the EfficientSU2 is the TwoLocal circuit. The TwoLocal also contains SU(2) blocks (or rotation blocks) and entanglement blocks. Here, we are free to specify the type of entanglement gates we want to use, for example CRX gates. In this example, all gates accept a parameter, but that need not be the case. One could use Y rotation gates, and CX entanglement gates, for example.

In [55]:
from qiskit.circuit.library import TwoLocal

rotation_blocks = ["ry"]
entanglement_blocks = ["crx"]
two_ansatz = TwoLocal(
    4, rotation_blocks, entanglement_blocks, "linear", insert_barriers=True, reps=2
)
two_ansatz.decompose().draw(output="mpl")

<Image src="/learning/images/courses/quantum-chem-with-vqe/ansatz/extracted-outputs/fdb6071a-7f29-4214-960c-93c304e64a6a-0.avif" alt="Output of the previous code cell" />

The last ansatz we will discuss by name is the PauliTwoDesign. This circuit contains an initial rotation of RY(pi/4), and rotation layers contain single qubit Pauli rotations, where the axis is chosen uniformly at random to be X, Y or Z. The entanglement layers are composed of pairwise CZ gates with a total depth of 2. Note the difference in entanglement (and total circuit) depth between this PauliTwoDesign and, for example, the EfficientSU2.

In [58]:
from qiskit.circuit.library import PauliTwoDesign

PtwoD_ansatz = PauliTwoDesign(5, reps=1, seed=10599, insert_barriers=True)
PtwoD_ansatz.decompose().draw(output="mpl")

<Image src="/learning/images/courses/quantum-chem-with-vqe/ansatz/extracted-outputs/1cff47db-5ee5-4e58-ac99-cbc2af9801c7-0.avif" alt="Output of the previous code cell" />

These pre-made variational circuits are useful heuristics both in terms of achieving a desired level of entanglement and in limiting circuit depth. But there is nothing magical about them. You are free to make your own variational circuit. Indeed that may be advantageous in cases where you know something about the entanglement of the target state of your system.

To make your own ansatz, you simply construct a quantum circuit, with some subset of gates being functions of elements of a parameter vector ("theta" in the 3-qubit example below).

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

n = 3

theta = ParameterVector("θ", length=n)
qc = QuantumCircuit(n)
qc.h(0)
qc.h(2)
for i in range(n - 1):
    qc.cx(i, i + 1)
qc.cz(0, n - 1)
qc.barrier()
for i in range(n):
    qc.ry(theta[i], i)
qc.barrier()
qc.cz(0, n - 1)
for i in reversed(range(n - 1)):
    qc.cx(i, i + 1)
qc.h(0)
qc.h(1)
own_ansatz = qc
print(own_ansatz.depth())
qc.draw("mpl")

9


<Image src="/learning/images/courses/quantum-chem-with-vqe/ansatz/extracted-outputs/1e7ad487-fc57-4c76-abb2-6279695fab5f-1.avif" alt="Output of the previous code cell" />

Generally, selecting the best ansatz is an art; the best ansatz is whatever ansatz helps you reach your target in the fewest optimization steps. It is easier to identify ansaetze that are likely to be bad. For example, greater circuit depth tends to result in accumulation of errors. Error mitigation can help with this, but it is a good practice to keep the circuit depth as low as is reasonable. But don't skip entanglement that is necessary. You might have a target state that requires a full entanglement scheme. Two examples are shown below that are likely to be poor choices for fairly obvious reasons. Choosing a good ansatz will be revisited in later sections in the context of convergence tests.

This first circuit is likely to be a poor choice because the last qubit is not entangled with the others at all. Indeed there is no computationally meaningful action on the last qubit at all. In all likelihood, the last qubit should either be entangled with the others or removed from the calculation.

In [ ]:
n = 4

theta = ParameterVector("θ", length=n)
qc = QuantumCircuit(n)
qc.h(0)
qc.h(2)
for i in range(n - 2):
    qc.cx(i, i + 1)
qc.cz(0, n - 2)
qc.barrier()
for i in range(n):
    qc.ry(theta[i], i)
qc.barrier()
qc.cz(0, n - 2)
for i in reversed(range(n - 2)):
    qc.cx(i, i + 1)
qc.h(0)
qc.h(1)
own_ansatz2 = qc
print(own_ansatz2.depth())
qc.draw("mpl")

9


<Image src="/learning/images/courses/quantum-chem-with-vqe/ansatz/extracted-outputs/f8ef9c33-1.avif" alt="Output of the previous code cell" />

This last circuit is likely a poor choice, because the gate depth is very high, and repeating the entanglement layer 4 times is not likely to yield a substantially better match to the target state than 2 or 3 reps.

In [66]:
SU2_ansatz_long = EfficientSU2(
    4, su2_gates=["rx", "y", "z"], entanglement="linear", reps=4
)
print(SU2_ansatz_long.decompose().depth())
SU2_ansatz_long.decompose().draw(output="mpl")

24


<Image src="/learning/images/courses/quantum-chem-with-vqe/ansatz/extracted-outputs/73627a8d-1.avif" alt="Output of the previous code cell" />

These circuits are not "complete" in the sense that there are still unknown and variable parameters to be inserted into many of the gates. Those parameters are chosen by making successive guesses, and updating parameters to lower the expectation value of the cost function (in the chemistry context, typically the ground state energy). In one, or even a few dimensions, this is trivial. But the circuit above has 20 variational parameters, meaning that finding the target state with the minimal energy means search a 20-dimensional state (another reasonn not to include unnecessary circuit gates). This is where classical optimizing algorithms come into play, and that is the subject of the next lesson.